In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('data/processed_data.csv', index_col=0)

In [4]:
X = df.loc[:,['return', 'gain_loss', 'macd', 'macd_diff_signal', \
              'diff_day', 'diff_10day', 'diff_ema12', 'diff_ema26',\
              'Exchange_NGM', 'Exchange_NMS', 'Exchange_NYQ', \
              'Sector_Communication Services', 'Sector_Consumer Cyclical', \
              'Sector_Consumer Defensive', 'Sector_Energy', \
              'Sector_Financial Services', 'Sector_Healthcare', 'Sector_Industrials', \
              'Sector_Real Estate', 'Sector_Technology', 'Sector_Utilities', \
              'cap_grouping_medium', 'cap_grouping_small']]
y = df.loc[:,['next_close']]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [7]:
from sklearn.metrics import r2_score

score = r2_score(y_pred, y_test)
print(score)

0.9988558070539748


In [8]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [9]:
score = r2_score(y_pred, y_test)
print(score)

0.9994422927460348


In [10]:
from sklearn.linear_model import SGDRegressor

model = SGDRegressor()
model.fit(X_train, y_train.to_numpy().ravel())

y_pred = model.predict(X_test)

In [11]:
score = r2_score(y_pred, y_test)
print(score)

-0.07054841789132649


In [12]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train.to_numpy().ravel())

y_pred = model.predict(X_test)

In [13]:
score = r2_score(y_pred, y_test)
print(score)

0.9993709727418416
